In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
load_dotenv()

True

In [3]:
key = os.getenv("OPENAI_KEY")

In [4]:
llm = ChatOpenAI(openai_api_key=key, model="gpt-3.5-turbo", temperature=0.7)

c:\Users\alams\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
with open("..\Response.json","r") as f:
    RESPONSE_JSON = json.load(f)

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\alams\AppData\Local\Temp\ipykernel_24396\1786520669.py:1: SyntaxWarning: invalid escape sequence '\R'
  with open("..\Response.json","r") as f:


In [6]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [7]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text. It is your job to \
create a quize of {number} multiple choice question for subject in {tone} tone.
Make sure the question are not repeated and check all the question confirming the text as well.
Make sure to format the reposne like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
    )

In [9]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt,output_key="quiz", verbose=True)

In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text","number","subject","tone","RESPONSE_JSON"], output_variables=["quiz","review"], verbose=True)

In [17]:
PATH = "..\src\data.txt"


<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\alams\AppData\Local\Temp\ipykernel_24396\513000508.py:1: SyntaxWarning: invalid escape sequence '\s'
  PATH = "..\src\data.txt"


In [18]:
print(PATH)

..\src\data.txt


In [21]:
with open(PATH,"r") as file:
    TEXT = file.read()

In [27]:
TEXT
NUMBER = 5
SUBJECT = "AI"
TONE ="Simple"
RESPONSE_JSON = RESPONSE_JSON


In [30]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [38]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }

    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large

In [37]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1421
Prompt Tokens:969
Completion Tokens:452
Total Cost:0.0023575000000000002


In [39]:
response

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:\n\nSystems, architecture and infrastructure for Generative AI\nMachine learning and Modeling using LLMs and Diffusion models\nLarge Language Models and its applications\nMulti-modal Generative AI and its applications\nGen AI b

In [41]:
quiz= response.get("quiz")

In [44]:
quiz = json.loads(quiz)

In [45]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct = value['correct']
    quiz_table_data.append({"MCQ":mcq,"Choices":options, "Correct":correct})

In [46]:
quiz_table_data

[{'MCQ': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a:Building robots | b:Developing self-driving cars | c:Building and deploying Generative AI systems | d:Creating virtual reality games',
  'Correct': 'c'},
 {'MCQ': 'Which topic is NOT mentioned as a focus area for Generative AI in the text?',
  'Choices': 'a:Responsible use of Gen AI | b:Evaluation of Language and Diffusion based models | c:Development of mobile applications | d:Systems, architecture, and infrastructure for Generative AI',
  'Correct': 'c'},
 {'MCQ': 'What type of models are mentioned in the text for Machine learning and Modeling?',
  'Choices': 'a:CNNs | b:LLMs and Diffusion models | c:Decision trees | d:Support Vector Machines',
  'Correct': 'b'},
 {'MCQ': 'What is one of the applications of Large Language Models mentioned in the text?',
  'Choices': 'a:Weather forecasting | b:Creating realistic text | c:Detecting fraud in financial transactions | d:Controlling traffic light

In [48]:
df = pd.DataFrame(quiz_table_data)

In [49]:
df.to_csv('genai.csv',index=False)

In [53]:
LOG_FILE = "{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [54]:
LOG_FILE

"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [55]:
from datetime import datetime

In [56]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_04_2024_00_47_26'